In [18]:
import requests
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats


with open("api_key.txt","r") as api_key_file:
    api_key = api_key_file.readline()


## Create DataFrame for the Top Rated Movies

Only really interested in getting the IDs for each movie

In [ ]:
list_of_top_movies = []

for page_num in range(1, 10):
    url = f"https://api.themoviedb.org/3/movie/top_rated?api_key={api_key}&page={page_num}"

    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers)
    json_response = response.json()

    for movie in json_response['results']:
        list_of_top_movies.append(movie)

top_movies = pd.DataFrame(list_of_top_movies)
top_movies

## Create DataFrame for the top rated movies with all detailed info

In [ ]:
list_of_detailed_movies = []

for index, row in top_movies.iterrows():
    id = row['id']

    url = f"https://api.themoviedb.org/3/movie/{id}?api_key={api_key}"

    headers = {
        "accept": "application/json",
    }

    response = requests.get(url, headers=headers)
    json_response = response.json()
    list_of_detailed_movies.append(json_response)

movies_detailed = pd.DataFrame(list_of_detailed_movies)

movies_detailed

## Create percent of revenue and compare with Runtime

In [ ]:
# look at adding new attribute that is normailzing budget and revenue | percent of revenue : ((revennue - budget) / budget) * 100
movies_detailed['percent_of_revenue'] = ((movies_detailed['revenue'] - movies_detailed['budget']) / movies_detailed['budget']) * 100
display(movies_detailed)
ax = sns.regplot(x='runtime', y='percent_of_revenue', data=movies_detailed, ci=None)
ax.set_title("Budget vs Runtime")
display(ax)